In [ ]:
!nvidia-smi

Wed Jul 24 04:56:34 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pwd

/content


In [ ]:
!rm -rf ultralytics
!git clone https://github.com/ultralytics/ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 30926, done.
remote: Counting objects: 100% (1372/1372), done.
remote: Compressing objects: 100% (837/837), done.
remote: Total 30926 (delta 913), reused 877 (delta 531), pack-reused 29554
Receiving objects: 100% (30926/30926), 15.81 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (22246/22246), done.


In [ ]:
!rm -rf yolov5
!ls

strawberry  ultralytics


In [ ]:
%pip install -qe ultralytics

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done


In [ ]:
%cd /content/ultralytics

/content/ultralytics


In [ ]:
!gsutil ls "gs://new_jeans/strawberry"

gs://new_jeans/strawberry/image/
gs://new_jeans/strawberry/label/
gs://new_jeans/strawberry/test_images/
gs://new_jeans/strawberry/test_labels/


In [ ]:
!gsutil -m cp -r "gs://new_jeans/strawberry" .

Copying gs://new_jeans/strawberry/image/train/black_strawberry 001.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 002.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 003.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 004.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 005.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 006.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 007.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 008.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 009.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 011.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 010.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 012.jpg...
Copying gs://new_jeans/strawberry/image/train/black_strawberry 013.jpg...
Copying gs://new_jeans/strawberry/imag

In [ ]:
from glob import glob

strawberry_img_path = "/content/ultralytics/strawberry/image/train/*.jpg"
strawberry_label_path = "/content/ultralytics/strawberry/label/train/*.txt"

strawberry_img_list = glob(strawberry_img_path)
strawberry_label_list = glob(strawberry_label_path)

len(strawberry_img_list), len(strawberry_label_list)

(842, 842)

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(
    strawberry_img_list,
    test_size = 0.2,
    random_state = 19
)

In [ ]:
train_txt = "/content/ultralytics/strawberry/train.txt"
val_txt = "/content/ultralytics/strawberry/val.txt"

In [ ]:
with open(train_txt, "w") as f:
  f.write("\n".join(train_img_list))

with open(val_txt, "w") as f:
  f.write("\n".join(val_img_list))

In [ ]:
!ls strawberry

image  label  test_images  test_labels	train.txt  val.txt


In [ ]:
import yaml

data = {
    "train": train_txt,
    "val": val_txt,
    "nc": 3,
    "name": ["rotten_strawberry", "sweet_strawberry", "unripe_strawberry"]
}

yaml_path = "/content/ultralytics/strawberry/data.yaml"

with open(yaml_path, "w") as f:
  yaml.dump(data, f)

In [ ]:
cfg_path = "/content/ultralytics/ultralytics/cfg/models/v8/yolov8.yaml"

In [ ]:
!cat {cfg_path} | head -n 5

# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes


In [ ]:
!sed -i "s/nc: 80/nc: 3/g" {cfg_path}

In [ ]:
!cat {cfg_path} | head -n 5

# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 3 # number of classes


In [ ]:
!pwd

/content/ultralytics


In [ ]:
!yolo \
task = detect \
mode = train \
model = yolov8m.pt \
data = {yaml_path} \
batch = 16 \
epochs = 10 \
imgsz = 640 \
name = strawberry_yolov8_weights \
exist_ok \
device = 0

In [ ]:
# best,pt
model = "/content/ultralytics/runs/detect/strawberry_yolov8_weights/weights/best.pt"
# 테스트 데이터
source = "https://youtu.be/Z8yvHueSq40?si=rG7XiXTMm4jTuMaZ" # 딸기영상 링크 넣으면 됨

In [ ]:
# 코드 실행 후 나오는 파일 경로에 들어가 영상 다운로드 후 확인해보기
!yolo \
task = detect \
mode = predict \
show = True \
model = {model} \
source = {source} \
imgsz = 640 \
name = "yolov8_predict" \
save=True \
exist_ok=True